In [207]:
import pandas as pd 
import numpy as np
import hashlib
from csv import reader
import re
import os
# pd.set_option('display.max_colwidth', None)

In [208]:
def hash_str(to_hash): 
    stringToHash = to_hash.encode()
    return hashlib.sha256(stringToHash).hexdigest()

In [209]:
def process_lbl(line_list, filename): 
    data_list = []
    i = 0
    name_line = ''
    while (i < len(line_list)): 
        if i+1 < len(line_list) and '---------,---------,---------,---------' in line_list[i+1]:
            name_line = line_list[i].strip('\n')     
        elif "---------,---------,---------,---------" in line_list[i]: 
            pass
        else: 
            for line in reader([filename+','+name_line+','+line_list[i].strip()]):
                data_list.append(line)
        i = i + 1
    return data_list        

In [210]:
table_file= open("table_test", "r")
table = '\n'.join([x for x in table_file])
table_file.close()

In [211]:
table

"\\begin{figure*}[t]\n\n    {\\footnotesize\n\n    \\centering\n\n        \\begin{tabularx}{\\textwidth}{\n\n            | >{\\centering\\arraybackslash}l|\n\n            >{\\centering\\arraybackslash}X\n\n            >{\\centering\\arraybackslash}X \n\n            >{\\centering\\arraybackslash}X \n\n            >{\\centering\\arraybackslash}X \n\n            >{\\centering\\arraybackslash}X \n\n            >{\\centering\\arraybackslash}X \n\n            >{\\centering\\arraybackslash}X \n\n            >{\\centering\\arraybackslash}X \n\n            >{\\centering\\arraybackslash}X\n\n            >{\\centering\\arraybackslash}X\n\n            ||} \n\n          \\textbf{Application} &\n\n            \\textbf{Document size} (B) &\n\n            \\textbf{\\# states} &\n\n            \\textbf{\\mbox{\\# transitions}} &\n\n            \\textbf{R1CS \\mbox{constraints}} &\n\n            \\textbf{\\# Steps} &\n\n            \\textbf{Compiler time} (s) &\n\n            \\textbf{Solver time} (s) &

In [212]:
data = []
directory = 'tests/results'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    if 'Data' not in filename and os.path.isfile(filepath):
        f = open(filepath, "r")
        lbl = [x for x in f]
        f.close()
        data.extend(process_lbl(lbl,filename))

In [213]:
df = pd.DataFrame(data, columns = ['app','doc','setup','time','regex','n_transitions','n_states',
                                    'test_type','component','test','val','metric'])
num_cols = ['time','n_transitions','n_states','val']
df[num_cols] = df[num_cols].apply(pd.to_numeric,errors='coerce', axis=1)

naive_sub = df.loc[(df.setup=='naive') 
       & (df.test.isin(['witness_generation','prove_0']))
      ].groupby(['time', 'regex','app','doc','setup','n_transitions','n_states','test_type']).sum().reset_index()
naive_sub['metric']='μs'
naive_sub['component'] = 'T'
naive_sub['test'] = 'prove+wit'
df = pd.concat([df,naive_sub])
df['hash_id'] = df.apply(lambda x: hash_str(x.doc+x.regex)[:5], axis=1)
df.loc[df.test=='prove+wit','component'] = 'T'

In [221]:
temp = df.loc[
    (df.app=='pihole') & 
    (df.metric=='μs') & 
    (df.test!="prove+wit") &
    (df.setup=='reef') & 
    (df.hash_id=='fcf51')].groupby(by=['time','component']).sum().reset_index()

In [222]:
temp.head()

,time,component,n_transitions,n_states,val
0,1700697598,C,20035,180,24430092
1,1700697598,CG,4007,36,686762
2,1700697598,P,16028,144,1836603
3,1700697598,S,12021,108,209718
4,1700697598,V,8014,72,412911


In [ ]:
pd.pivot_table(temp, )

In [201]:
(df.loc[(df.app=='pihole') 
       & (df.metric=='μs') 
       & (df.test=="prove+wit")].groupby(by=['regex','setup']).std().val/df.loc[
    (df.app=='pihole') 
    & (df.metric=='μs') 
    & (df.test=="prove+wit")].groupby(by=['regex','setup']).mean().val)*100

regex                                                                                  setup       
^(.+[_.-])?adse?rv(er?|ice)?s?[0-9]*[_.-]                                              naive           0.687925
                                                                                       nwr             0.584612
                                                                                       reef            3.195497
                                                                                       safa+nlookup    1.997862
^(.+[_.-])?telemetry[_.-]                                                              naive           0.298998
                                                                                       nwr             0.651170
                                                                                       reef            1.962772
                                                                                       safa+nlookup    1.514250
^ad(

In [199]:
df.loc[(df.app=='pihole') & (df.metric=='μs') & (df.test=="prove+wit")].groupby(by=['regex','setup']).mean().val

regex                                                                                  setup       
^(.+[_.-])?adse?rv(er?|ice)?s?[0-9]*[_.-]                                              naive            18787006.6
                                                                                       nwr              48600932.3
                                                                                       reef              1995571.6
                                                                                       safa+nlookup      1968505.2
^(.+[_.-])?telemetry[_.-]                                                              naive            10551759.7
                                                                                       nwr              42901848.0
                                                                                       reef              1596751.7
                                                                                       safa+nlo

In [161]:
df.loc[(df.app=='pihole') & (df.metric=='μs') & (df.test=="prove+wit")].groupby(by=['regex','setup']).var()/1e6

val
regex                                                                                 setup                      
^(.+[_.-])?adse?rv(er?|ice)?s?[0-9]*[_.-]                                             nwr            80727.967525
                                                                                      reef            4066.411863
                                                                                      safa+nlookup    1546.692280
^(.+[_.-])?telemetry[_.-]                                                             nwr            78044.202670
                                                                                      reef             982.232551
                                                                                      safa+nlookup     579.242881
^ad([sxv]?[0-9]*|system)[_.-]([^.[:space:]]+\.){1,}|[_.-]ad([sxv]?[0-9]*|system)[_.-] nwr           203997.244248
                                                                                      reef            3034.654088
                                                                                      safa+nlookup    3244.378238
^adim(age|g)s?[0-9]*[_.-]                                                             nwr           181331.210487
                                                                                      reef            3297.437841
                                                                                      safa+nlookup     517.831524
^adtrack(er|ing)?[0-9]*[_.-]                                                          nwr           105924.777146
                                                                                      reef            1452.280015
                                                                                      safa+nlookup    1111.536812
^advert(s|is(ing|ements?))?[0-9]*[_.-]                                                nwr            86990.646591
                                                                                      reef             696.236094
                                                                                      safa+nlookup     128.992361
^aff(iliat(es?|ion))?[_.-]                                                            nwr            77261.436944
                                                                                      reef            2505.888913
                                                                                      safa+nlookup    1879.906302
^analytics?[_.-]                                                                      nwr            71996.378023
                                                                                      reef             786.685033
                                                                                      safa+nlookup    1720.746114
^banners?[_.-]                                                                        nwr            85478.450238
                                                                                      reef            1010.452535
                                                                                      safa+nlookup    2114.356407
^beacons?[0-9]*[_.-]                                                                  nwr           139054.849786
                                                                                      reef            1668.038727
                                                                                      safa+nlookup    2669.213159
^count(ers?)?[0-9]*[_.-]                                                              nwr            39270.816296
                                                                                      reef            1846.467144
                                                                                      safa+nlookup    1518.331564
^mads\.                                                                               nwr           130859.016473
      